In [26]:
import pandas as pd

In [27]:
from collections import Counter

In [28]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [145]:
c = pd.read_csv('311requests.csv')

In [148]:
c.columns = ['date', 'agency', 'category', 'description', 'zip', 'address', 'street', 'city', 'boro', 'x', 'y']

In [150]:
len(c.index)

2285043

In [151]:
c['date'] = pd.to_datetime(c.date.values)

In [152]:
c['category'] = c['category'].str.lower()

In [153]:
c['description'] = c['description'].str.lower()

In [154]:
c.ix[c.category.str.contains('street sign'), 'category'] = 'street sign'

In [155]:
c.ix[c.category.str.contains('highway sign'), 'category'] = 'highway sign'

In [156]:
c.ix[c.category.str.contains('fire alarm'), 'category'] = 'fire alarm'

In [157]:
c = c.replace({'derelict vehicles': 'derelict vehicle', 'public assembly - temporary': 'public assembly', 'scrie': 'dof literature request', 'broken parking meter': 'broken muni meter'})

In [158]:
c = c.sort_values(by = 'date')

In [159]:
len(c)

2285043

In [166]:
full = c[pd.notnull(c['x']) & pd.notnull(c['y'])]

In [167]:
len(full)

2050027

In [168]:
full.to_pickle('full.pkl')

In [175]:
for_gis = full[['x', 'y']].reset_index().copy()

In [178]:
for_gis.drop('index', axis=1, inplace=True)

In [180]:
for_gis.to_csv('for_gis.csv')

#### get acs

In [10]:
dem = pd.read_csv('/home/amn34/metis/stuff/acs/demo.csv',usecols=['GeogName', 'GeoID', 'MaleP','MdAgeE','WtNHP','Pop18plE'], thousands=',')

In [11]:
dem.head()

,GeogName,GeoID,MaleP,MdAgeE,Pop18plE,WtNHP
0,Brooklyn Heights-Cobble Hill ...,BK09,49.4,36.9,20645.0,76.2
1,Sheepshead Bay-Gerritsen Beach-Manhattan Beac ...,BK17,45.9,43.8,51145.0,69.8
2,Brighton Beach ...,BK19,46.9,44.5,26813.0,69.2
3,Seagate-Coney Island ...,BK21,45.8,42.0,22492.0,30.2
4,West Brighton ...,BK23,40.8,57.2,13908.0,97.3


In [12]:
econ = pd.read_csv('/home/amn34/metis/stuff/acs/econ.csv', usecols=['CvLFUEmP', 'Inc_SNAPP', 'FamBwPvP','MdHHIncE'], thousands=',')

In [13]:
hous = pd.read_csv('/home/amn34/metis/stuff/acs/housing.csv', usecols=['HUE','VacHUP','BltBf39P','ROcHUP','MdGRE'], thousands=',')

In [14]:
soc = pd.read_csv('/home/amn34/metis/stuff/acs/social.csv', usecols=['HHE', 'FamP', 'SmHsP', 'NtvP'], thousands=',')

In [15]:
acs = pd.concat([dem, econ, hous, soc], axis=1)

In [16]:
acs.dropna(inplace=True)

In [17]:
acs.head()

,GeogName,GeoID,MaleP,MdAgeE,Pop18plE,WtNHP,CvLFUEmP,MdHHIncE,Inc_SNAPP,FamBwPvP,HUE,VacHUP,BltBf39P,ROcHUP,MdGRE,HHE,FamP,SmHsP,NtvP
0,Brooklyn Heights-Cobble Hill ...,BK09,49.4,36.9,20645.0,76.2,4.7,105126.0,2.4,2.8,12760.0,12.1,66.7,55.6,1893.0,11222.0,42.4,83.0,80.5
1,Sheepshead Bay-Gerritsen Beach-Manhattan Beac ...,BK17,45.9,43.8,51145.0,69.8,5.4,50721.0,17.7,13.5,27855.0,8.0,26.4,47.5,1077.0,25614.0,65.7,93.7,54.7
2,Brighton Beach ...,BK19,46.9,44.5,26813.0,69.2,4.2,29690.0,31.2,20.1,15518.0,9.0,35.5,70.5,1043.0,14122.0,56.2,88.4,31.1
3,Seagate-Coney Island ...,BK21,45.8,42.0,22492.0,30.2,10.2,23324.0,50.3,28.0,12101.0,10.2,16.3,83.4,619.0,10863.0,61.7,92.1,64.4
4,West Brighton ...,BK23,40.8,57.2,13908.0,97.3,4.3,40285.0,24.9,8.2,8641.0,6.3,4.8,58.7,905.0,8094.0,53.6,96.8,39.2


In [18]:
acs.to_pickle('acs.pkl')

#### nta

In [22]:
full = pd.read_pickle('full.pkl')

In [24]:
full[full.category=='srde']

,date,agency,category,description,zip,address,street,city,boro,x,y
1570710,2015-03-23 12:52:00,DEP,srde,NaN,11692,72-03 HILLMEYER AVENUE,HILLMEYER AVENUE,Arverne,QUEENS,1039380.0,156980.0
917087,2015-04-14 16:54:00,DEP,srde,NaN,11357,NaN,NaN,Whitestone,QUEENS,1032243.0,227363.0
1394866,2015-05-12 21:00:00,DEP,srde,NaN,10314,NaN,NaN,STATEN ISLAND,STATEN ISLAND,930900.0,155420.0
1395098,2015-05-19 19:09:00,DEP,srde,NaN,10314,250 ASHWORTH AVE,ASHWORTH AVE,STATEN ISLAND,STATEN ISLAND,942708.0,155065.0
1383723,2015-07-26 18:26:00,DEP,srde,NaN,10464,196 HORTON ST,HORTON ST,BRONX,BRONX,1044556.0,245394.0
1368323,2015-07-30 10:47:00,DEP,srde,NaN,10314,773 MANOR ROAD,MANOR ROAD,STATEN ISLAND,STATEN ISLAND,950638.0,160734.0
1365648,2015-11-12 18:23:00,DEP,srde,NaN,10302,NaN,NaN,STATEN ISLAND,STATEN ISLAND,946155.0,166745.0


In [219]:
f = full.sort_values(by = 'date').reset_index()

In [215]:
nta = pd.read_csv('/home/amn34/metis/stuff/311/pts/ptswithnta.csv', usecols=['NTACode'])

In [223]:
recode = pd.concat([f, nta], axis=1)

In [224]:
recode.head()

,index,date,agency,category,description,zip,address,street,city,boro,x,y,NTACode
0,418928,2015-01-01 00:00:50,NYPD,blocked driveway,no access,11420,123-19 135 STREET,135 STREET,SOUTH OZONE PARK,QUEENS,1038733.0,184971.0,QN55
1,258068,2015-01-01 00:01:29,NYPD,blocked driveway,no access,10466,931 EAST 226 STREET,EAST 226 STREET,BRONX,BRONX,1024816.0,262237.0,BX44
2,980466,2015-01-01 00:01:30,NYPD,noise - street/sidewalk,loud music/party,10031,508 WEST 139 STREET,WEST 139 STREET,NEW YORK,MANHATTAN,997847.0,238629.0,MN04
3,418819,2015-01-01 00:04:28,NYPD,noise - vehicle,car/truck horn,10468,2555 SEDGWICK AVENUE,SEDGWICK AVENUE,BRONX,BRONX,1009923.0,255465.0,BX30
4,1108415,2015-01-01 00:04:44,NYPD,illegal parking,blocked hydrant,11421,84-25 85 ROAD,85 ROAD,WOODHAVEN,QUEENS,1022809.0,192567.0,QN53


In [226]:
recode[recode.NTACode=='QN72']

,index,date,agency,category,description,zip,address,street,city,boro,x,y,NTACode
172,2013713,2015-01-01 01:06:19,NYPD,noise - residential,loud music/party,11105,20-44 20 STREET,20 STREET,Astoria,QUEENS,1008021.0,224828.0,QN72
520,352252,2015-01-01 03:13:28,TLC,taxi complaint,driver complaint,11105,29-19 21 AVENUE,21 AVENUE,ASTORIA,QUEENS,1009486.0,222833.0,QN72
770,2013772,2015-01-01 05:02:07,NYPD,noise - residential,loud music/party,11105,42-09 ASTORIA BOULEVARD,ASTORIA BOULEVARD,ASTORIA,QUEENS,1009798.0,219472.0,QN72
819,2014008,2015-01-01 05:26:44,NYPD,noise - residential,loud music/party,11105,42-09 ASTORIA BOULEVARD,ASTORIA BOULEVARD,ASTORIA,QUEENS,1009798.0,219472.0,QN72
875,1407610,2015-01-01 06:15:00,DEP,sewer,manhole cover broken/making noise (sb),10014,535 HUDSON ST,HUDSON ST,NEW YORK,MANHATTAN,982539.0,206891.0,QN72
936,13008,2015-01-01 07:19:34,NYPD,illegal parking,posted parking sign violation,11105,23-04 36 STREET,36 STREET,ASTORIA,QUEENS,1008972.0,220651.0,QN72
1583,389926,2015-01-01 15:30:40,TLC,taxi complaint,driver complaint,11105,23-24 28 STREET,28 STREET,ASTORIA,QUEENS,1007690.0,221600.0,QN72
1671,363502,2015-01-01 16:45:06,NYPD,illegal parking,blocked hydrant,11105,22-22 38 STREET,38 STREET,ASTORIA,QUEENS,1009908.0,220882.0,QN72
2011,1407766,2015-01-01 20:22:00,DOT,street light condition,street light out,11105,NaN,NaN,Astoria,QUEENS,1007727.0,222018.0,QN72
2466,420168,2015-01-02 00:00:00,HPD,heat/hot water,entire building,10031,523 WEST 138 STREET,WEST 138 STREET,NEW YORK,MANHATTAN,997541.0,238509.0,QN72


In [209]:
new=new.drop(['index', 'street', 'field_1'], axis=1)

In [228]:
new.columns

Index([u'date', u'agency', u'category', u'description', u'zip', u'address',
       u'city', u'boro', u'x', u'y', u'X', u'Y', u'x', u'y', u'BoroCode',
       u'BoroName', u'CountyFIPS', u'NTACode', u'NTAName', u'Shape_Leng',
       u'Shape_Area'],
      dtype='object')

In [229]:
new.to_pickle('new.pkl')

In [230]:
newshort = new.drop(['X', 'Y', 'BoroCode', 'BoroName', 'CountyFIPS', 'Shape_Leng', 'Shape_Area'], axis=1)

In [232]:
newshort.columns = ['date', 'agency', 'category', 'description', 'zip', 'address', 'city', 'boro', 'x', 'y', 'x1', 'y1', 'ntacode', 'ntaname']

In [235]:
newshort.drop(['x1', 'y1'], axis=1, inplace=True)

/home/amn34/.local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [237]:
newshort.to_pickle('newshort.pkl')

In [20]:
parks = ['BK99', 'BX99', 'MN99', 'SI99', 'QN99', 'BX98', 'QN98']

In [239]:
len(newshort)

2050027

In [243]:
noparks = newshort.drop(newshort[newshort.ntacode.isin(parks)].index)

In [244]:
len(noparks)

2040860

In [247]:
noparks.to_pickle('noparks.pkl')

#### check categories

In [29]:
c[c.category=='building/use']

NameError: name 'c' is not defined

In [182]:
light = c[c.category=='street light condition']

In [230]:
light

,date,agency,type,description,loctype,zip,address,city,boro,coord,month
374227,2015-01-01 01:03:00,DOT,street light condition,street light out,NaN,11103,42-11 25 AVENUE,Astoria,QUEENS,"(40.76720500546349, -73.91008705244731)",1
599886,2015-01-01 04:45:00,DOT,street light condition,lamppost knocked down,NaN,11375,NaN,Forest Hills,QUEENS,"(40.715118540089534, -73.83223298198979)",1
374245,2015-01-01 06:31:00,DOT,street light condition,street light out,NaN,11233,455 MACON STREET,BROOKLYN,BROOKLYN,"(40.6827628163424, -73.93500689426507)",1
374247,2015-01-01 10:01:00,DOT,street light condition,street light out,NaN,11385,57-13 CATALPA AVENUE,Ridgewood,QUEENS,"(40.70073874581195, -73.90418991508835)",1
599943,2015-01-01 10:03:00,DOT,street light condition,lamppost knocked down,NaN,NaN,H HUDSON PKWY,NaN,BRONX,NaN,1
599961,2015-01-01 10:42:00,DOT,street light condition,lamppost knocked down,NaN,NaN,132-026 CONDUIT AVE S,NaN,QUEENS,NaN,1
374122,2015-01-01 11:35:00,DOT,street light condition,street light out,NaN,11201,146 MONTAGUE STREET,BROOKLYN,BROOKLYN,"(40.69427601992895, -73.99242349482047)",1
374128,2015-01-01 11:39:00,DOT,street light condition,street light out,NaN,11224,1214 SURF AVENUE,BROOKLYN,BROOKLYN,"(40.57556543881814, -73.9812021930983)",1
374071,2015-01-01 12:09:00,DOT,street light condition,street light out,NaN,11219,NaN,BROOKLYN,BROOKLYN,"(40.63572632638595, -73.98421895439142)",1
598281,2015-01-01 12:32:00,DOT,street light condition,street light out,NaN,NaN,NaN,NaN,BROOKLYN,NaN,1


In [191]:
noaddress = c[c['address'].isnull() & c['coord'].isnull()]

In [192]:
len(noaddress)

161153

In [197]:
noaddress[noaddress.category=='traffic signal condition']

,date,agency,type,description,loctype,zip,address,city,boro,coord,month
374076,2015-01-01 01:00:00,DOT,traffic signal condition,led pedestrian unit,NaN,NaN,NaN,NaN,BRONX,NaN,1
374089,2015-01-01 01:50:00,DOT,traffic signal condition,controller,NaN,NaN,NaN,NaN,BROOKLYN,NaN,1
374275,2015-01-01 02:10:00,DOT,traffic signal condition,veh signal head,NaN,NaN,NaN,NaN,BRONX,NaN,1
373993,2015-01-01 02:56:00,DOT,traffic signal condition,controller,NaN,NaN,NaN,NaN,BRONX,NaN,1
374235,2015-01-01 04:30:00,DOT,traffic signal condition,led pedestrian unit,NaN,NaN,NaN,NaN,MANHATTAN,NaN,1
373714,2015-01-01 07:15:00,DOT,traffic signal condition,vehicle signal,NaN,NaN,NaN,NaN,MANHATTAN,NaN,1
374107,2015-01-01 08:00:00,DOT,traffic signal condition,led pedestrian unit,NaN,NaN,NaN,NaN,MANHATTAN,NaN,1
374063,2015-01-01 08:20:00,DOT,traffic signal condition,veh signal head,NaN,NaN,NaN,NaN,MANHATTAN,NaN,1
371503,2015-01-01 09:58:00,DOT,traffic signal condition,controller,NaN,NaN,NaN,NaN,BROOKLYN,NaN,1
374070,2015-01-01 10:00:00,DOT,traffic signal condition,veh signal head,NaN,NaN,NaN,NaN,BRONX,NaN,1


In [200]:
DOT = c[c.agency=='DOT']

In [202]:
Counter(DOT.category).most_common()

[('street condition', 124934),
 ('street light condition', 98106),
 ('traffic signal condition', 42210),
 ('broken muni meter', 22972),
 ('highway condition', 7020),
 ('sidewalk condition', 5416),
 ('street sign - damaged', 5156),
 ('street sign - missing', 4332),
 ('dot literature request', 2913),
 ('street sign - dangling', 1868),
 ('curb condition', 1584),
 ('broken parking meter', 791),
 ('bridge condition', 498),
 ('ferry inquiry', 479),
 ('ferry complaint', 297),
 ('bus stop shelter placement', 198),
 ('agency issues', 163),
 ('bike rack condition', 134),
 ('municipal parking facility', 98),
 ('public toilet', 58),
 ('ferry permit', 56),
 ('highway sign - damaged', 49),
 ('parking card', 41),
 ('highway sign - missing', 39),
 ('highway sign - dangling', 15),
 ('tunnel condition', 12),
 ('building condition', 3),
 ('opinion for the mayor', 2)]

In [226]:
check = 'sewer'
len(c[c.category==check]), len(noaddress[noaddress.category==check]), len(nocoords[nocoords.category==check])

(32111, 892, 231)

In [203]:
Counter(noaddress.category).most_common()

[('dof literature request', 29690),
 ('benefit card replacement', 22464),
 ('street light condition', 21803),
 ('traffic signal condition', 17026),
 ('hpd literature request', 11498),
 ('opinion for the mayor', 10290),
 ('agency issues', 8395),
 ('maintenance or facility', 8275),
 ('scrie', 7797),
 ('street condition', 2951),
 ('dot literature request', 2913),
 ('highway condition', 2334),
 ('dca / doh new license application request', 1807),
 ('animal in a park', 1408),
 ('water system', 1363),
 ('violation of park rules', 1337),
 ('dpr internal', 1003),
 ('sewer', 892),
 ('dca literature request', 871),
 ('sanitation condition', 784),
 ('request for information', 613),
 ('taxi compliment', 594),
 ('noise', 514),
 ('ferry inquiry', 479),
 ('dirty conditions', 458),
 ('ferry complaint', 299),
 ('derelict vehicles', 287),
 ('snow', 282),
 ('dep literature request', 201),
 ('broken muni meter', 175),
 ('damaged tree', 173),
 ('bridge condition', 166),
 ('oem literature request', 163),
 (

In [175]:
nocoords = c[c['address'].notnull() & c['coord'].isnull()]

In [186]:
len(nocoords)

32796

In [228]:
nclights = nocoords[nocoords.type=='street light condition']

In [229]:
nclights[nclights.address.str.contains('PARK')]

,date,agency,type,description,loctype,zip,address,city,boro,coord,month
599938,2015-01-01 21:23:00,DOT,street light condition,street light out,NaN,NaN,GRAVESEND PARK,NaN,BROOKLYN,NaN,1
600125,2015-01-01 21:23:00,DOT,street light condition,street light out,NaN,NaN,GRAVESEND PARK,NaN,BROOKLYN,NaN,1
598295,2015-01-01 21:24:00,DOT,street light condition,street light out,NaN,NaN,GRAVESEND PARK,NaN,BROOKLYN,NaN,1
597946,2015-01-01 21:24:00,DOT,street light condition,street light out,NaN,NaN,GRAVESEND PARK,NaN,BROOKLYN,NaN,1
599815,2015-01-01 21:25:00,DOT,street light condition,street light out,NaN,NaN,GRAVESEND PARK,NaN,BROOKLYN,NaN,1
600082,2015-01-01 21:26:00,DOT,street light condition,street light out,NaN,NaN,GRAVESEND PARK,NaN,BROOKLYN,NaN,1
600271,2015-01-01 21:26:00,DOT,street light condition,street light out,NaN,NaN,GRAVESEND PARK,NaN,BROOKLYN,NaN,1
600272,2015-01-01 21:26:00,DOT,street light condition,street light out,NaN,NaN,GRAVESEND PARK,NaN,BROOKLYN,NaN,1
375154,2015-01-02 08:58:00,DOT,street light condition,time clock maladjusted,NaN,NaN,MARINE PARK,NaN,BROOKLYN,NaN,1
375162,2015-01-02 08:58:00,DOT,street light condition,street light out,NaN,NaN,MARINE PARK,NaN,BROOKLYN,NaN,1


In [188]:
Counter(nocoords.category).most_common()

[('street light condition', 17625),
 ('street condition', 2830),
 ('illegal parking', 1576),
 ('taxi complaint', 962),
 ('general construction/plumbing', 956),
 ('noise - street/sidewalk', 855),
 ('consumer complaint', 582),
 ('building/use', 557),
 ('graffiti', 499),
 ('noise', 470),
 ('noise - commercial', 454),
 ('sanitation condition', 450),
 ('water system', 436),
 ('blocked driveway', 378),
 ('broken muni meter', 360),
 ('street sign - missing', 265),
 ('sewer', 231),
 ('derelict vehicle', 229),
 ('noise - vehicle', 225),
 ('homeless person assistance', 209),
 ('taxi report', 195),
 ('for hire vehicle complaint', 185),
 ('street sign - damaged', 168),
 ('dirty conditions', 135),
 ('elevator', 117),
 ('noise - park', 113),
 ('rodent', 92),
 ('maintenance or facility', 91),
 ('special projects inspection team (spit)', 89),
 ('plumbing', 89),
 ('electrical', 67),
 ('sidewalk condition', 63),
 ('air quality', 63),
 ('derelict vehicles', 60),
 ('overgrown tree/branches', 56),
 ('other

In [144]:
c.date.values[1]

numpy.datetime64('2015-01-01T00:01:29.000000000')

In [151]:
c['month'] = [pd.Timestamp(i).month for i in c.date.values]

In [80]:
categories = set(c.category.values)

In [19]:
len(categories)

NameError: name 'categories' is not defined

In [82]:
categories

{'adopt-a-basket',
 'advocate - other',
 'advocate - rpie',
 'advocate-commercial exemptions',
 'advocate-personal exemptions',
 'advocate-prop class incorrect',
 'advocate-scrie/drie',
 'agency',
 'agency issues',
 'air quality',
 'animal abuse',
 'animal facility - no permit',
 'animal in a park',
 'appliance',
 'asbestos',
 'beach/pool/sauna complaint',
 'benefit card replacement',
 'best/site safety',
 'bike rack condition',
 'bike/roller/skate chronic',
 'blocked driveway',
 'boilers',
 'bottled water',
 'bridge condition',
 'broken muni meter',
 'broken parking meter',
 'building condition',
 'building/use',
 'bus stop shelter placement',
 'calorie labeling',
 'city vehicle placard complaint',
 'collection truck noise',
 'complaint',
 'compliment',
 'construction',
 'consumer complaint',
 'cranes and derricks',
 'curb condition',
 'damaged tree',
 'day care',
 'dca / doh new license application request',
 'dca literature request',
 'dead tree',
 'dep literature request',
 'dereli

In [248]:
c[c.category=='unspecified']

,date,agency,type,description,loctype,zip,address,city,boro,coord,month
1266071,2015-10-07 14:38:37,DOHMH,unspecified,unspecified,Unspecified,10468,2430 MORRIS AVENUE,BRONX,MANHATTAN,"(40.86136267480809, -73.90006493958619)",10
185320,2015-10-22 11:51:48,DOB,unspecified,unspecified,Unspecified,NaN,111 LEXINGTON AVENUE,NaN,MANHATTAN,NaN,10


In [29]:
housing = c[c.agency == 'HPD']

In [30]:
set(housing.category.values)

{'AGENCY',
 'APPLIANCE',
 'Agency Issues',
 'DOOR/WINDOW',
 'ELECTRIC',
 'ELEVATOR',
 'FLOORING/STAIRS',
 'GENERAL',
 'HEAT/HOT WATER',
 'HPD Literature Request',
 'OUTSIDE BUILDING',
 'PAINT/PLASTER',
 'PLUMBING',
 'SAFETY',
 'UNSANITARY CONDITION',
 'VACANT APARTMENT',
 'WATER LEAK'}

In [83]:
catcount = Counter(c.category.values)

In [25]:
catcount.most_common()

NameError: name 'catcount' is not defined

In [37]:
agencies = set(c.agency.values)

In [38]:
len(agencies)

47

In [39]:
agencycount = Counter(c.agency.values)

In [40]:
agencycount.most_common()

[('HPD', 627715),
 ('NYPD', 364558),
 ('DOT', 319444),
 ('DEP', 185641),
 ('DSNY', 155283),
 ('DOB', 88581),
 ('DPR', 75541),
 ('DOHMH', 58856),
 ('DOF', 41000),
 ('HRA', 27367),
 ('TLC', 26521),
 ('DCA', 20364),
 ('CHALL', 10305),
 ('FDNY', 6906),
 ('DHS', 6574),
 ('DOE', 1636),
 ('EDC', 1505),
 ('3-1-1', 1294),
 ('DOITT', 430),
 ('DFTA', 394),
 ('NYCEM', 164),
 ('OATH', 118),
 ('DOP', 43),
 ('MOVA', 10),
 ('OPS', 9),
 ('MOIA', 8),
 ('MOFTB', 7),
 ('NYCHA', 7),
 ('DOC', 5),
 ('MOPD', 5),
 ('ACS', 4),
 ('OCME', 4),
 ('AJC', 3),
 ('FUND', 3),
 ('LAW', 3),
 ('DCAS', 2),
 ('NYCPPF', 2),
 ('NYCOOA', 2),
 ('NYCSERVICE', 2),
 ('COIB', 1),
 ('LPC', 1),
 ('SBS', 1),
 ('CAU', 1),
 ('CCRB', 1),
 ('DYCD', 1),
 ('OPA', 1),
 ('MOC', 1)]

In [ ]:
vector = c[c.category == 'vector']

In [130]:
Counter(vector.description)

Counter({'Basement': 7,
         'Bird Bath': 1,
         'Building Foundation': 18,
         'Container - Over 5 Gallons': 34,
         'Container - Under 5 Gallons': 7,
         'Flower Planters': 2,
         'Fountain - Over 5 Gallons': 1,
         'Fountain - Under 5 Gallons': 1,
         'Other - Explain Below': 173,
         'Puddle in Ground': 57,
         'Puddle on Driveway': 3,
         'Puddle on Roof': 12,
         'Puddle on Sidewalk': 14,
         'Roof Gutters': 5,
         'Sewer or Drain': 19,
         'Swimming Pool - Unmaintained': 182,
         'Swimming Pool Cover': 17,
         'Tires': 20})

In [125]:
sanitation = c[c.category=='sanitation condition']

In [126]:
Counter(sanitation.description)

Counter({'12 Dead Animals': 8964,
         '15 Street Cond/Dump-Out/Drop-Off': 21532,
         '15R Street Cond/Ref.W Door': 114,
         '22 Weeds': 1854,
         '9 Spill/Oil etc': 1})

In [123]:
dirty = c[c.category=='dirty conditions']

In [124]:
Counter(dirty.description)

Counter({'E1 Improper Disposal': 3254,
         'E11 Litter Surveillance': 1304,
         'E12 Illegal Dumping Surveillance': 3039,
         'E13 Throw-Out': 187,
         'E15 Illegal Postering': 4116,
         'E1A Litter Basket / Improper Use': 334,
         'E2 Receptacle Violation': 1326,
         'E3 Dirty Sidewalk': 10111,
         'E3A Dirty Area/Alleyway': 6122,
         'E3B Sidewalk Obstruction': 93,
         'E4 18" Law': 598,
         'E5 Loose Rubbish': 1283,
         'E7 Private Carter Spillage': 32,
         'E8 Canine Violation': 2635})

In [112]:
pets = c[c.category=='illegal animal kept as pet']

In [113]:
Counter(pets.description)

Counter({'Farm Animal': 55,
         'Ferret': 6,
         'Iguana': 1,
         'Monkey': 2,
         'Other': 77,
         'Rooster': 127,
         'Snake': 18})